# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np
import os
import pandas as pd

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import warnings
warnings.filterwarnings("ignore")

### SELECT DEVICE ###
# GPU device configuration
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
  print('Using CUDA')
elif torch.backends.mps.is_available():
  DEVICE = torch.device('mps')
  print('Using MPS')
else:
  DEVICE = torch.device('cpu')
  print('Using CPU')

Using CUDA


# Dataloader

In [2]:
### DEFINE TRANSFORMATIONS ###
normalizer = transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                  std = [0.229, 0.224, 0.225])
train_transforms = transforms.Compose([
                    transforms.Resize((224,224)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    normalizer
                    ])

# train_transforms = transforms.Compose([
#                                         transforms.Lambda(lambda img: img.crop((20, 0, 150, 130))),  # Crop
#                                         transforms.Resize((224,224)),
#                                         transforms.Lambda(lambda img: img.convert("RGB")),
#                                         transforms.RandomHorizontalFlip(),
#                                         transforms.ToTensor(),
#                                         normalizer
#                                       ])

class GuessTheCorrelationDataset(Dataset):
  def __init__(self, root, transform=None, indexes=None):
    self.root = root
    self.transform = transform
    self.img_dir = os.path.join(root, 'train_imgs')
    
    # Load correlation values
    csv_path = os.path.join(root, 'train_responses.csv')
    df = pd.read_csv(csv_path)
    
    # Filter by indexes if provided
    if indexes is not None:
      df = df.loc[df.index.isin(indexes)]
    
    self.img_files = df['id'].astype(str) + '.png'
    self.correlations = df['corr'].values
  
  def __len__(self):
    return len(self.img_files)
  
  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_files.iloc[idx])
    image = Image.open(img_path)
    
    if self.transform:
      image = self.transform(image)
    
    label = torch.tensor(self.correlations[idx], dtype=torch.float32)
    
    return image, label

In [3]:
if os.name == 'posix':
  root = os.path.expanduser("~/Documents/PyTorch_Data/correlation/guess-the-correlation")
else:
  root = "d:\\PyTorch_Data\\guess-the-correlation"
  
train_imgs_path = os.path.join(root, 'train_imgs')
# num_images = len([name for name in os.listdir(train_imgs_path) if os.path.isfile(os.path.join(train_imgs_path, name))])
# print(f"Number of images: {num_images}")

# Model training function

In [4]:
def train(model, device, epochs, optimizer, loss_fn, batch_size, trainloader, valloader):
    log_training = {"epoch": [],
                    "training_loss": [],
                    "training_acc": [],
                    "validation_loss": [],
                    "validation_acc": []}

    for epoch in range(1, epochs + 1):
        print(f"Starting Epoch {epoch}")
        training_losses, training_accuracies = [], []
        validation_losses, validation_accuracies = [], []

        for image, label in tqdm(trainloader):
            image, label = image.to(device), label.to(device)
            optimizer.zero_grad()
            out = model.forward(image)
        
            ### CALCULATE LOSS ##
            loss = loss_fn(out, label)
            training_losses.append(loss.item())

            ### CALCULATE ACCURACY ###
            predictions = torch.argmax(out, axis=1)
            accuracy = (predictions == label).sum() / len(predictions)
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()

        for image, label in tqdm(valloader):
            image, label = image.to(device), label.to(device)
            with torch.no_grad():
                out = model.forward(image)

                ### CALCULATE LOSS ##
                loss = loss_fn(out, label)
                validation_losses.append(loss.item())

                ### CALCULATE ACCURACY ###
                predictions = torch.argmax(out, axis=1)
                accuracy = (predictions == label).sum() / len(predictions)
                validation_accuracies.append(accuracy.item())

        training_loss_mean, training_acc_mean = np.mean(training_losses), np.mean(training_accuracies)
        valid_loss_mean, valid_acc_mean = np.mean(validation_losses), np.mean(validation_accuracies)

        log_training["epoch"].append(epoch)
        log_training["training_loss"].append(training_loss_mean)
        log_training["training_acc"].append(training_acc_mean)
        log_training["validation_loss"].append(valid_loss_mean)
        log_training["validation_acc"].append(valid_acc_mean)

        print("Training Loss:", training_loss_mean) 
        print("Training Acc:", training_acc_mean)
        print("Validation Loss:", valid_loss_mean)
        print("Validation Acc:", valid_acc_mean)
        print("=====================================\n")
        
    return log_training, model

## Load PreTrained Weights but Only Train the Final Classifier Layer

In [5]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model.fc
model.fc = nn.Linear(2048, 1)

# Check the name of all the parameters
for name, param in model.named_parameters():
  # print(name)
  if "fc" not in name:
    param.requires_grad_(False) # Inplace turn of gradient updates

Using cache found in C:\Users\middletonk/.cache\torch\hub\pytorch_vision_v0.10.0


In [6]:
total_parameters = 0
for name, params in model.named_parameters():
  num_params = int(torch.prod(torch.tensor(params.shape)))
  print(name, ":", params.shape, "Num Parameters:", num_params)
  total_parameters += num_params

print("------------------------")
print("Total Parameters in Model", total_parameters)

conv1.weight : torch.Size([64, 3, 7, 7]) Num Parameters: 9408
bn1.weight : torch.Size([64]) Num Parameters: 64
bn1.bias : torch.Size([64]) Num Parameters: 64
layer1.0.conv1.weight : torch.Size([64, 64, 1, 1]) Num Parameters: 4096
layer1.0.bn1.weight : torch.Size([64]) Num Parameters: 64
layer1.0.bn1.bias : torch.Size([64]) Num Parameters: 64
layer1.0.conv2.weight : torch.Size([64, 64, 3, 3]) Num Parameters: 36864
layer1.0.bn2.weight : torch.Size([64]) Num Parameters: 64
layer1.0.bn2.bias : torch.Size([64]) Num Parameters: 64
layer1.0.conv3.weight : torch.Size([256, 64, 1, 1]) Num Parameters: 16384
layer1.0.bn3.weight : torch.Size([256]) Num Parameters: 256
layer1.0.bn3.bias : torch.Size([256]) Num Parameters: 256
layer1.0.downsample.0.weight : torch.Size([256, 64, 1, 1]) Num Parameters: 16384
layer1.0.downsample.1.weight : torch.Size([256]) Num Parameters: 256
layer1.0.downsample.1.bias : torch.Size([256]) Num Parameters: 256
layer1.1.conv1.weight : torch.Size([64, 256, 1, 1]) Num Para

In [7]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
model.fc = nn.Linear(2048, 1)

# Check the name of all the parameters
for name, param in model.named_parameters():
  # print(name)
  if "fc" not in name:
    param.requires_grad_(False) # Inplace turn of gradient updates

model = model.to(DEVICE)

### MODEL TRAINING INPUTS ###
epochs = 5
optimizer = optim.SGD(params=model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss()
batch_size = 1

# Dataset creation
train_ds = GuessTheCorrelationDataset(root, transform=train_transforms, indexes=range(300))
valid_ds = GuessTheCorrelationDataset(root, transform=train_transforms, indexes=range(300, 400))
test_ds = GuessTheCorrelationDataset(root, transform=train_transforms, indexes=range(400, 500))

### BUILD DATALOADERS ###
trainloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=1)
valloader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=1)
testloader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=1)

Using cache found in C:\Users\middletonk/.cache\torch\hub\pytorch_vision_v0.10.0


In [10]:
for i in range(len(trainloader)):
    try:
        if i % 10 == 0:
            print(f"Batch {i} loaded successfully")
        data_iterator = iter(trainloader)
        data = next(data_iterator)
    except Exception as e:
        print(f"Error at batch {i}: {e}")

Batch 0 loaded successfully
Error at batch 0: DataLoader worker (pid(s) 30476) exited unexpectedly
Error at batch 1: DataLoader worker (pid(s) 24744) exited unexpectedly
Error at batch 2: DataLoader worker (pid(s) 2292) exited unexpectedly
Error at batch 3: DataLoader worker (pid(s) 5032) exited unexpectedly
Error at batch 4: DataLoader worker (pid(s) 34456) exited unexpectedly
Error at batch 5: DataLoader worker (pid(s) 10616) exited unexpectedly


KeyboardInterrupt: 

In [8]:
data_iterator = iter(trainloader)
data = next(data_iterator)
# data
# features, labels = data
# print(features, "\n", labels)

RuntimeError: DataLoader worker (pid(s) 21260) exited unexpectedly

In [ ]:

random_init_logs, model = train(model=model,
                                device=DEVICE,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=trainloader,
                                valloader=valloader)